<a href="https://colab.research.google.com/github/QuantAnalyticsTorch/quant_analytics_torch/blob/main/examples/WienerPathTensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Upgrade to TensorFlow nightly
#!pip install --upgrade tf-nightly-cpu

In [76]:
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

Python version
3.7.3 (v3.7.3:ef4ec6ed12, Mar 25 2019, 22:22:05) [MSC v.1916 64 bit (AMD64)]
Version info.
sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)


In [77]:
import tensorflow as tf
from scipy.special import erf
import numpy as np
import time
import math

In [78]:
SQRT_2 = tf.sqrt(tf.constant(2.,dtype=tf.float64))

def norminv(x):
    return SQRT_2*tf.math.erfinv(2*(x-0.5))

In [173]:
class UnivariateBrownianBridge():
  def __init__(self, number_time_steps):
    self.number_time_steps = number_time_steps

    self.left_index = np.zeros(number_time_steps, dtype=int)
    self.right_index = np.zeros(number_time_steps, dtype=int)
    self.bridge_index = np.zeros(number_time_steps, dtype=int)
    self.left_weight = np.zeros(number_time_steps)
    self.right_weight = np.zeros(number_time_steps)
    self.std_dev = np.zeros(number_time_steps)

    self._map = np.zeros(number_time_steps, dtype=int)

    self._map[-1] = 1
    self.bridge_index[0] = number_time_steps - 1
    self.std_dev[0] = math.sqrt(1.0 * number_time_steps)
    self.left_weight[0] = 0
    self.right_weight[0] = 0

    j=0
    for i in range(1,number_time_steps):
      while self._map[j] == True:
        j = j + 1
      k = j
      while self._map[k] == False:
        k = k + 1
      l = j+((k-1-j)>>1)
      self._map[l]=i
      self.bridge_index[i]=l
      self.left_index[i]=j
      self.right_index[i]=k
      self.left_weight[i]=(k-l)/(k+1-j)
      self.right_weight[i]=(1+l-j)/(k+1-j)
      self.std_dev[i]=np.sqrt(((1+l-j)*(k-l))/(k+1-j))
      j=k+1
      if j>=number_time_steps:
        j=0

    self.bridge_index_t = tf.constant(tf.convert_to_tensor(self.bridge_index,dtype=tf.int32))
    self.right_index_t = tf.constant(tf.convert_to_tensor(self.right_index,dtype=tf.int32))
    self.left_index_t = tf.constant(tf.convert_to_tensor(self.left_index,dtype=tf.int32))
    self.std_dev_t = tf.constant(tf.convert_to_tensor(self.std_dev))
    self.right_weight_t = tf.constant(tf.convert_to_tensor(self.right_weight))
    self.left_weight_t = tf.constant(tf.convert_to_tensor(self.left_weight))

  @tf.function(jit_compile=True, input_signature=(tf.TensorSpec(shape=(None,None), dtype=tf.float64),
                                                tf.TensorSpec(shape=(), dtype=tf.bool),
                                                tf.TensorSpec(shape=(), dtype=tf.int32),
                                                tf.TensorSpec(shape=(None), dtype=tf.int32),
                                                tf.TensorSpec(shape=(None), dtype=tf.int32),
                                                tf.TensorSpec(shape=(None), dtype=tf.int32),
                                                tf.TensorSpec(shape=(None), dtype=tf.float64),
                                                tf.TensorSpec(shape=(None), dtype=tf.float64),
                                                tf.TensorSpec(shape=(None), dtype=tf.float64)))
  def buildPath(z, increment, number_time_steps, left_index, right_index, bridge_index, left_weight, right_weight, std_dev):
    path = tf.TensorArray(dtype=tf.float64,size=number_time_steps+1)
    path = path.write(number_time_steps,std_dev[0]*z[0])
    path = path.write(0,0*z[0]);  
    j = 0
    k = 0
    l = 0
    i = 0
    for i in range(1,number_time_steps):
      j = left_index[i]
      k = right_index[i]
      l = bridge_index[i]
      path = path.write(l+1,left_weight[i] * path.read(j) + right_weight[i] * path.read(k+1) + std_dev[i] * z[i])

    if increment:
      for i in range(0,number_time_steps):
        path = path.write(i, path.read(i+1) - path.read(i))

    return path.stack()

  def path(self,z,increment):
    return UnivariateBrownianBridge.buildPath(z, increment, self.number_time_steps, self.left_index_t, self.right_index_t, self.bridge_index_t, self.left_weight_t, self.right_weight_t, self.std_dev_t)

In [174]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
dim =  1#@param {type:"integer"}
number =  7#@param {type:"integer"}

brownian = UnivariateBrownianBridge(dim)

x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
z = tf.transpose(norminv(x))

print(z)
print(brownian.path(z, False))
print(brownian.path(z, True))

tf.Tensor(
[[ 0.          0.67448975 -0.67448975 -0.31863936  1.15034938  0.31863936
  -1.15034938]], shape=(1, 7), dtype=float64)
tf.Tensor(
[[ 0.          0.         -0.         -0.          0.          0.
  -0.        ]
 [ 0.          0.67448975 -0.67448975 -0.31863936  1.15034938  0.31863936
  -1.15034938]], shape=(2, 7), dtype=float64)
tf.Tensor(
[[ 0.          0.67448975 -0.67448975 -0.31863936  1.15034938  0.31863936
  -1.15034938]
 [ 0.          0.67448975 -0.67448975 -0.31863936  1.15034938  0.31863936
  -1.15034938]], shape=(2, 7), dtype=float64)


In [175]:
#@title Pricing time a CPU. Note TensorFlow does automatic multithreading.
dim =  128#@param {type:"integer"}
number =  200000#@param {type:"integer"}

brownian = UnivariateBrownianBridge(dim)

time_start = time.time()
with tf.device("/cpu:0"):
    x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
    z = tf.transpose(norminv(x))
    w = brownian.path(z, True)
time_end = time.time()
time_price_cpu = time_end - time_start
print("Runtime on CPU: ", time_price_cpu)

Runtime on CPU:  0.6209940910339355


In [177]:
dim =  256#@param {type:"integer"}
number =  100000#@param {type:"integer"}

time_start = time.time()

brownian = UnivariateBrownianBridge(dim)

with tf.profiler.experimental.Profile("logs/brownian"):
  with tf.device('/cpu'):
    x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
    z = tf.transpose(norminv(x))
    w = brownian.path(z, True)
time_end = time.time()
time_price_tpu = time_end - time_start
print("Time on a CPU: ", time_price_tpu)    

Time on a CPU:  0.6091873645782471


In [165]:
!pip install -U tensorboard_plugin_profile

You are using pip version 19.0.3, however version 20.3b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
Requirement already up-to-date: tensorboard_plugin_profile in d:\home\quant_analytics_torch\venv\lib\site-packages (2.3.0)


In [166]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [198]:
S = 1
K = 1
sigma = 0.2
sdt = np.sqrt(1./dim)

p = tf.TensorArray(dtype=tf.float64,size=1)
p = p.write(0,S*tf.ones(shape=(number),dtype=tf.float64));  

In [199]:
brownian = UnivariateBrownianBridge(dim)

x = tf.math.sobol_sample(dim,number,dtype=tf.dtypes.float64)
z = tf.transpose(norminv(x))
w = brownian.path(z, True)

In [200]:
for i in range(dim):
    p = p.write(0, p.read(0) * (1. + sigma * sdt * w[i]))

In [201]:
x = p.read(0)

In [202]:
tf.math.reduce_mean(tf.math.maximum(x-1,0))

<tf.Tensor: shape=(), dtype=float64, numpy=0.07965404472410584>

In [203]:
w = brownian.path(z, False)

In [160]:
w[-1]

<tf.Tensor: shape=(100000,), dtype=float64, numpy=
array([  0.        ,  10.791836  , -10.791836  , ...,   1.82671272,
       -27.05099133, -24.73321531])>